In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from scipy.stats import zscore
from scipy import stats
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/content/StudentsPerformance.csv')
print(df.head())

   gender race/ethnicity parental level of education         lunch  \
0  female        group B           bachelor's degree      standard   
1  female        group C                some college      standard   
2  female        group B             master's degree      standard   
3    male        group A          associate's degree  free/reduced   
4    male        group C                some college      standard   

  test preparation course  math score  reading score  writing score  \
0                    none          72             72             74   
1               completed          69             90             88   
2                    none          90             95             93   
3                    none          47             57             44   
4                    none          76             78             75   

  performance  
0        pass  
1        pass  
2        pass  
3        fail  
4        pass  


In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race/ethnicity               1000 non-null   object
 2   parental level of education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test preparation course      1000 non-null   object
 5   math score                   1000 non-null   int64 
 6   reading score                1000 non-null   int64 
 7   writing score                1000 non-null   int64 
 8   performance                  1000 non-null   object
dtypes: int64(3), object(6)
memory usage: 70.4+ KB
None


In [ ]:
#Dropping Unnecessary Columns
X2 = df.drop(columns=['lunch'])
y2 = df['lunch']

In [ ]:
#Remove duplicates
duplicate_rows = df.duplicated().sum()
print(f"\nNumber of duplicate rows: {duplicate_rows}")


Number of duplicate rows: 0


In [ ]:
#encode categorical variables
for col in df.select_dtypes(include='object').columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
df['gender_encoded'] = le.fit_transform(df['gender'])
print(df[['gender_encoded']])

     gender_encoded
0                 0
1                 0
2                 0
3                 1
4                 1
..              ...
995               0
996               1
997               0
998               0
999               0

[1000 rows x 1 columns]


In [ ]:
#Identify numeric columns
numeric_cols_2 = df.select_dtypes(include=['int64', 'float64']).columns

In [ ]:
scaler_minmax = MinMaxScaler()
df_minmax = df.copy()
df_minmax[numeric_cols_2] = scaler_minmax.fit_transform(df[numeric_cols_2])
df_minmax


,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,performance,gender_encoded
0,0.0,0.25,0.2,1.0,1.0,0.72,0.662651,0.711111,1.0,0.0
1,0.0,0.50,0.8,1.0,0.0,0.69,0.879518,0.866667,1.0,0.0
2,0.0,0.25,0.6,1.0,1.0,0.90,0.939759,0.922222,1.0,0.0
3,1.0,0.00,0.0,0.0,1.0,0.47,0.481928,0.377778,0.0,1.0
4,1.0,0.50,0.8,1.0,1.0,0.76,0.734940,0.722222,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
995,0.0,1.00,0.6,1.0,0.0,0.88,0.987952,0.944444,1.0,0.0
996,1.0,0.50,0.4,0.0,1.0,0.62,0.457831,0.500000,1.0,1.0
997,0.0,0.50,0.4,0.0,0.0,0.59,0.650602,0.611111,1.0,0.0
998,0.0,0.75,0.8,1.0,0.0,0.68,0.734940,0.744444,1.0,0.0


In [ ]:
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,performance,gender_encoded
0,0,1,1,1,1,72,72,74,1,0
1,0,2,4,1,0,69,90,88,1,0
2,0,1,3,1,1,90,95,93,1,0
3,1,0,0,0,1,47,57,44,0,1
4,1,2,4,1,1,76,78,75,1,1


#Feature Extraction

##Automated feature extraction


In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Calculate subject ratios (makes PCA more interpretable)
df['math_to_verbal'] = df['math score'] / (df['reading score'] + df['writing score'])
df['reading_to_writing'] = df['reading score'] / df['writing score']

# Standardize and apply PCA
scaler = StandardScaler()
pca_features = scaler.fit_transform(df[['math score', 'reading score', 'writing score',
                                       'math_to_verbal', 'reading_to_writing']])

pca = PCA(n_components=2)
components = pca.fit_transform(pca_features)

# Name the components based on loadings
df['overall_ability'] = components[:, 0]  # Positive weights on all scores
df['math_specialization'] = components[:, 1]  # High weight on math-to-verbal ratio

print("PCA Interpretation Guide:")
print("1. overall_ability: Higher values = better at all subjects")
print(df['overall_ability'])

print("2. math_specialization: Positive = math strength, Negative = verbal strength", 'math_specialization')
print(df['math_specialization'])

PCA Interpretation Guide:
1. overall_ability: Higher values = better at all subjects
0      0.682394
1      1.701479
2      2.820039
3     -2.805929
4      0.915362
         ...   
995    2.931056
996   -1.151321
997   -0.474483
998    0.791947
999    1.802566
Name: overall_ability, Length: 1000, dtype: float64
2. math_specialization: Positive = math strength, Negative = verbal strength math_specialization
0      0.041974
1     -1.437069
2      0.088021
3      0.311044
4      0.302779
         ...   
995   -0.258293
996    1.015500
997   -0.590625
998   -0.679829
999   -0.528566
Name: math_specialization, Length: 1000, dtype: float64


In [ ]:
# Identify subject weaknesses
df['biggest_gap'] = df[['math score', 'reading score', 'writing score']].std(axis=1)
df['weak_subject'] = df[['math score', 'reading score', 'writing score']].idxmin(axis=1)
print(df['weak_subject'])


0         math score
1         math score
2         math score
3      writing score
4      writing score
           ...      
995       math score
996    reading score
997       math score
998       math score
999       math score
Name: weak_subject, Length: 1000, dtype: object


In [ ]:
# Target-encode support factors relative to performance
support_factors = ['lunch', 'test preparation course', 'parental level of education']
for factor in support_factors:
    # Calculate average performance per category
    means = df.groupby(factor)['overall_ability'].mean()
    df[f'{factor}_support'] = df[factor].map(means)
print(df[['lunch_support', 'test preparation course_support', 'parental level of education_support']])

     lunch_support  test preparation course_support  \
0         0.359253                        -0.355416   
1         0.359253                         0.637365   
2         0.359253                        -0.355416   
3        -0.652728                        -0.355416   
4         0.359253                        -0.355416   
..             ...                              ...   
995       0.359253                         0.637365   
996      -0.652728                        -0.355416   
997      -0.652728                         0.637365   
998       0.359253                         0.637365   
999      -0.652728                        -0.355416   

     parental level of education_support  
0                               0.539694  
1                               0.096321  
2                               0.736997  
3                               0.214443  
4                               0.096321  
..                                   ...  
995                             0.7369

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Step 1: Create total and subject-ratio features
df['total_score'] = df['math score'] + df['reading score'] + df['writing score']
df['math_ratio'] = df['math score'] / df['total_score']  # Math emphasis
df['verbal_ratio'] = (df['reading score'] + df['writing score']) / df['total_score']  # Verbal emphasis

# Verify calculation
print(df[['math score', 'reading score', 'writing score', 'total_score']].head(3))

   math score  reading score  writing score  total_score
0          72             72             74          218
1          69             90             88          247
2          90             95             93          278


In [ ]:
# Prepare features for PCA (standardize first)
features = ['total_score', 'math_ratio', 'verbal_ratio']
X = df[features]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA
pca = PCA(n_components=2)
components = pca.fit_transform(X_scaled)

# Create named components
df['academic_strength'] = components[:, 0]  # Primary component (overall performance)
df['subject_balance'] = components[:, 1]    # Secondary component (math vs. verbal)

# Interpretation guide
print("\nPCA Interpretation:")
print(f"Component 1 (academic_strength) weights: {pca.components_[0]}")
print(f"Component 2 (subject_balance) weights: {pca.components_[1]}")
print("\n• academic_strength ≈ Overall performance (higher = better)")
print("• subject_balance > 0 ≈ Math-oriented, < 0 ≈ Verbally-oriented")
print(df[['academic_strength', 'subject_balance']])


PCA Interpretation:
Component 1 (academic_strength) weights: [-0.06409984 -0.70565261  0.70565261]
Component 2 (subject_balance) weights: [ 0.99794349 -0.04532544  0.04532544]

• academic_strength ≈ Overall performance (higher = better)
• subject_balance > 0 ≈ Math-oriented, < 0 ≈ Verbally-oriented
     academic_strength  subject_balance
0            -0.249914         0.328230
1             1.800439         1.139679
2            -0.071206         1.746090
3             0.377553        -1.272246
4            -0.332299         0.580775
..                 ...              ...
995           0.403227         1.870323
996          -1.422280        -0.825300
997           0.923989        -0.135481
998           0.784613         0.511878
999           0.568650         1.107438

[1000 rows x 2 columns]


In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

feature_cols = ['gender_encoded', 'overall_ability', 'math_specialization',
                'biggest_gap', 'lunch_support',
                'test preparation course_support', 'parental level of education_support',
                'academic_strength', 'subject_balance']

X = df[feature_cols]
y = df['lunch']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

classifiers = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "SVC": SVC(random_state=42),
    "KNeighbors": KNeighborsClassifier(),
    "Gaussian Naive Bayes": GaussianNB(),
    "MLP Classifier": MLPClassifier(random_state=42, max_iter=1000)
}

results = {}

for name, clf in classifiers.items():
    clf.fit(X_train_scaled, y_train)
    y_pred = clf.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)
    results[name] = {
        "accuracy": accuracy,
        "report": report
    }
    print(f"--- {name} ---")
    print(f"Accuracy: {accuracy:.4f}")
    print(classification_report(y_test, y_pred))
    print("-" * (len(name) + 6))


--- Logistic Regression ---
Accuracy: 1.0000
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        94
           1       1.00      1.00      1.00       156

    accuracy                           1.00       250
   macro avg       1.00      1.00      1.00       250
weighted avg       1.00      1.00      1.00       250

-------------------------
--- Decision Tree ---
Accuracy: 1.0000
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        94
           1       1.00      1.00      1.00       156

    accuracy                           1.00       250
   macro avg       1.00      1.00      1.00       250
weighted avg       1.00      1.00      1.00       250

-------------------
--- Random Forest ---
Accuracy: 1.0000
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        94
           1       1.00      1.00      1.00       156

    accuracy 